In [73]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import statsmodels.api as sm
from sklearn.cluster import KMeans
from factor_analyzer import FactorAnalyzer


In [74]:
start_year = 2024 # for testing 2022
start = f'{start_year}-01-02'
SP500_list = list(pd.read_csv("../../constituents.csv")['Symbol'])
df_close = yf.download(SP500_list[:500], start=start, end=pd.to_datetime(start)+timedelta(days=1), progress=False)
df_close_t = df_close['Volume'].T
df_close_t.columns = ['Volume']
df_close_t.dropna(inplace=True)
list_ = df_close_t.sort_values(by='Volume', ascending=False)[:100].index.to_list()
list_[:5], len(list_)


2 Failed downloads:
['SOLV', 'GEV']: Exception("%ticker%: Data doesn't exist for startDate = 1704171600, endDate = 1704258000")


(['TSLA', 'AAPL', 'AMD', 'PFE', 'T'], 100)

In [75]:
v4_Antony_Factor2 = {
        "Equity" : ["VTI","IWB","SOXX","XLI","XLY","VIOV","XLE","DIA","VOX","XLP","XLK","XLF","QQQ","VOO"],
        "Bond/Fixed Income":["AGG", "BND", "BNDX", "TLT", "VCIT"],
        "Commodity":["COMT", "FTGC", "DBC", "GSG", "DJP"],
        "Currency":["UUP", "USDU", "FXY", "FXE", "FXC"],
        "Specialty-leveraged":["UDOW", "SOXL", "TQQQ", "TMF", "UPRO"],
        "Specialty-inverse":["SQQQ", "SH", "SDS", "SOXS", "SPXU"],
        "Factor":["AVUV", "AVDV", "MTUM", "IMTM", "QVAL", "IVAL", "QMOM", "IMTM"],
        "Other":[],
        "Other_2":[]}
def get_etf_list(etf_list):
    all_etf_list = []
    for etf in etf_list.values():
        all_etf_list.extend(etf)
    return all_etf_list
etfs = get_etf_list(v4_Antony_Factor2)

In [76]:
stock_list = list_
# stock_list = etfs

df_close = yf.download(stock_list, start=pd.to_datetime(start) - timedelta(days=365*5), end=start, progress=False)['Adj Close']
spx_close = yf.download("^GSPC", start=pd.to_datetime(start) - timedelta(days=365*5), end=start, progress=False)['Adj Close']
df_close.head()

Ticker,AAL,AAPL,ABBV,AMAT,AMCR,AMD,AMZN,BA,BAC,BAX,...,VICI,VTRS,VZ,WBA,WBD,WDC,WFC,WMB,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-03,29.581667,34.029236,67.046135,29.739157,7.308957,17.049999,75.014000,302.100525,21.593487,58.967354,...,14.274738,24.113369,42.255066,53.084671,25.680000,33.835331,39.721035,16.357409,28.289970,52.220562
2019-01-04,31.530161,35.481926,69.206154,31.775827,7.308957,19.000000,78.769501,317.822571,22.490284,60.814648,...,14.434445,25.539091,42.360291,54.850750,26.139999,35.268463,40.898079,17.037455,28.466671,54.145924
2019-01-07,32.425678,35.402954,70.216248,32.341557,7.466137,20.570000,81.475502,318.823425,22.472700,60.512871,...,14.472469,26.344936,42.630863,55.166115,26.530001,36.521271,40.633671,17.471531,28.801786,54.427494
2019-01-08,31.904110,36.077843,70.542564,31.030933,7.418983,20.750000,82.829002,330.891937,22.428738,60.348251,...,14.913565,26.451202,43.878529,56.269924,27.240000,36.170109,40.548378,17.992422,29.002855,54.823223
2019-01-09,32.888199,36.690502,68.227142,32.341557,7.426843,20.190001,82.971001,334.098511,22.648542,61.381664,...,15.111297,25.627647,43.326405,56.947964,27.280001,37.764595,40.770134,18.194994,28.908413,55.112408


In [77]:
df_close_pct = df_close.pct_change().iloc[1:, :]
spx_pct = spx_close.pct_change()

df_close_pct.dropna(inplace=True, axis=1)
spx_pct.dropna(inplace=True)

df_close_pct_m = df_close_pct.resample("ME").agg(lambda x: (x+1).prod() - 1)
spx_pct_m = spx_pct.resample("ME").agg(lambda x: (x+1).prod() - 1)

In [78]:
factor = 9
res_df = pd.DataFrame()
alpha_dict = {}
X = spx_pct_m
X_sm = sm.add_constant(X)
for ticker in df_close_pct_m.columns:
    # OLS regression
    close = df_close_pct_m[ticker]
    y = close
    model = sm.OLS(y, X_sm)

    res = model.fit()
    residuals = res.resid
    res_df[ticker] = residuals

    # store alpha
    res_table = res.summary().tables
    alpha = float(res_table[1].data[1][1].strip())
    alpha_dict[ticker] = alpha
factor_analyzer = FactorAnalyzer(n_factors=factor, rotation='promax') # varimax
factor_analyzer.fit(res_df.corr())

# Get factor loadings
factor_loadings = factor_analyzer.loadings_
df = pd.DataFrame(factor_loadings, index=list(res_df.columns))
df['Ticker'] = df.index
# Apply K-means clustering
kmeans = KMeans(n_clusters=factor)
kmeans.fit(df.iloc[:, :factor])

# Assign cluster labels to tickers
df['Cluster'] = kmeans.labels_

# Display the result
res = df[['Ticker', 'Cluster']]
res = res.copy()
# add clustered value to check further
cluster_value = []
for i in range(len(res)):
    classification = res.iloc[i, 1]
    cluster_value.append(df.iloc[i, classification])
res['Cluster_value'] = cluster_value

sorted_res = res.sort_values(by='Cluster')


res_dict = {}
for i in range(factor):

    res_dict["class" + str(i)] = list(sorted_res[sorted_res['Cluster'] == i]['Ticker'])
res_dict
# factor == 3比較穩定 但還是不合理

{'class0': ['PG',
  'KO',
  'PPL',
  'EXC',
  'CL',
  'T',
  'CAG',
  'NEM',
  'PFE',
  'BSX',
  'MRK',
  'VZ',
  'MNST',
  'AMCR',
  'MDLZ',
  'WMT',
  'JNJ',
  'BAX',
  'PEP'],
 'class1': ['MU', 'GM', 'FCX', 'F', 'MCHP', 'WDC', 'TXN', 'AMAT', 'WBD', 'ON'],
 'class2': ['RF',
  'WFC',
  'LUV',
  'LVS',
  'USB',
  'UAL',
  'NCLH',
  'TFC',
  'SCHW',
  'RTX',
  'MS',
  'KIM',
  'AAL',
  'DIS',
  'DAL',
  'JPM',
  'HPQ',
  'HPE',
  'FIS',
  'HBAN',
  'BA',
  'BAC',
  'CCL',
  'KEY',
  'C'],
 'class3': ['PYPL',
  'QCOM',
  'NVDA',
  'NEE',
  'GOOG',
  'MSFT',
  'TSLA',
  'GOOGL',
  'META',
  'AMZN',
  'AMD',
  'AAPL',
  'MTCH'],
 'class4': ['SLB',
  'CVX',
  'WMB',
  'PARA',
  'XOM',
  'OXY',
  'KMI',
  'PCG',
  'MRO',
  'DVN'],
 'class5': ['SBUX', 'CMCSA', 'NKE', 'ORCL', 'INTC'],
 'class6': ['MRNA', 'CSCO', 'FTNT', 'ABBV'],
 'class7': ['KDP', 'WBA', 'VTRS', 'KHC', 'MO', 'BMY', 'CVS', 'GILD'],
 'class8': ['JCI', 'CSX', 'VICI', 'O']}

In [79]:
import json
with open(f"farrell_for{start_year}_top100_cluster9.json", "w") as f:
    json.dump(res_dict, f, indent=4)

In [80]:
# classification index
import numpy as np
indexes = pd.DataFrame()
for key, value in res_dict.items():
    print(value)
    index_close = df_close[value]
    index_close = index_close/len(value)
    indexes[f"{str(key)}-index"] = index_close.sum(axis=1)

indexes_pct = indexes.pct_change()
indexes_pct.dropna(inplace=True)

indexes_pct_m = indexes_pct.resample("ME").agg(lambda x: (x+1).prod() - 1)
mean = np.mean(indexes_pct_m, axis=0)
sd = np.std(indexes_pct_m, axis=0)
reg_coef = []
for i in range(len(indexes_pct_m.columns)):
    X = spx_pct_m
    X_sm = sm.add_constant(X)
    y = indexes_pct_m.iloc[:, i]
    model = sm.OLS(y, X_sm)
    res = model.fit()
    reg_coef.append(res.summary().tables[1].data[2][1].replace(" ", ""))


result = pd.DataFrame()
result.index = list(indexes.columns)
result["mean"] = mean
result["sd"] = sd
result["reg_coef"] = reg_coef
result.sort_values(by='reg_coef', ascending=False)


['PG', 'KO', 'PPL', 'EXC', 'CL', 'T', 'CAG', 'NEM', 'PFE', 'BSX', 'MRK', 'VZ', 'MNST', 'AMCR', 'MDLZ', 'WMT', 'JNJ', 'BAX', 'PEP']
['MU', 'GM', 'FCX', 'F', 'MCHP', 'WDC', 'TXN', 'AMAT', 'WBD', 'ON']
['RF', 'WFC', 'LUV', 'LVS', 'USB', 'UAL', 'NCLH', 'TFC', 'SCHW', 'RTX', 'MS', 'KIM', 'AAL', 'DIS', 'DAL', 'JPM', 'HPQ', 'HPE', 'FIS', 'HBAN', 'BA', 'BAC', 'CCL', 'KEY', 'C']
['PYPL', 'QCOM', 'NVDA', 'NEE', 'GOOG', 'MSFT', 'TSLA', 'GOOGL', 'META', 'AMZN', 'AMD', 'AAPL', 'MTCH']
['SLB', 'CVX', 'WMB', 'PARA', 'XOM', 'OXY', 'KMI', 'PCG', 'MRO', 'DVN']
['SBUX', 'CMCSA', 'NKE', 'ORCL', 'INTC']
['MRNA', 'CSCO', 'FTNT', 'ABBV']
['KDP', 'WBA', 'VTRS', 'KHC', 'MO', 'BMY', 'CVS', 'GILD']
['JCI', 'CSX', 'VICI', 'O']


,mean,sd,reg_coef
class1-index,0.020399,0.094386,1.4488
class2-index,0.004898,0.087704,1.3132
class3-index,0.024749,0.081700,1.2774
class4-index,0.012417,0.108635,1.2501
class8-index,0.011114,0.068357,1.0765
class5-index,0.011138,0.065122,1.0355
class6-index,0.023203,0.118870,1.0172
class0-index,0.007723,0.040420,0.5367
class7-index,0.003233,0.048450,0.5256


In [81]:
# check for correlation after auto factorized
import numpy as np
cov = res_df.corr()
for cls in res_dict.values():
    res_df_temp = res_df[cls]
    corr = res_df_temp.corr()
    list_corr = []
    for i in range(len(corr)):
        for j in range(i):
            list_corr.append(corr.iloc[i, j])
    print(np.mean(list_corr))
# delet anything with correlation < 0.15

0.24770211112684837
0.34398009573103633
0.4025859879112524
0.17157232075309212
0.5283543857442476
0.09030308234413163
0.2121161817832964
0.2297649492742557
0.31963210012158444


In [82]:
cov = res_df.corr()
# correlation matrix cleaning
for i in range(len(cov)):
    for j in range(i+1):
        cov.iloc[i, j] = 0
classi = 3
for i in range(3):
    max_value = cov.max().max()  # Get the maximum value across all rows and columns
    max_row, max_col = cov.stack().idxmax()  
    print(max_value, max_row, max_col)

    cov.drop([max_row, max_col], inplace=True)
    cov.drop(columns=[max_row, max_col], inplace=True)

0.994121085551782 GOOG GOOGL
0.9263683994063718 DVN MRO
0.8808621789589095 HBAN RF
